In [1]:
import os

In [2]:
os.getcwd()

'/Users/reetkanjilal/Kaggel/Home_Credit_Default'

In [4]:
import pandas as pd
import numpy as np

train = pd.DataFrame(pd.read_csv('application_train.csv'))
test = pd.DataFrame(pd.read_csv('application_test.csv'))

In [15]:
pd.value_counts(train['TARGET'])

0    282686
1     24825
Name: TARGET, dtype: int64

In [16]:
24825/282686

0

In [9]:
print ('Train Size:',train.shape)

('Train Size:', (307511, 122))


In [12]:
train_sample = train.sample(frac=0.01)
test_sample = test.sample(frac=0.01)

print ('Train Sample Size: ', train_sample.shape)
print ('Test Sample Size: ',test_sample.shape )

('Train Sample Size: ', (3075, 122))
('Test Sample Size: ', (487, 121))


In [13]:
train_raw = train_sample.copy()
test_raw = test_sample.copy()

from sklearn import preprocessing
# Create a label encoder object
le = preprocessing.LabelEncoder()
le_count = 0

# Iterate through the columns
for col in train_raw:
    if train_raw[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train_raw[col].unique())) <= 2:
            # Train on the training data
            le.fit(train_raw[col])
            # Transform both training and testing data
            train_raw[col] = le.transform(train_raw[col])
            test_raw[col] = le.transform(test_raw[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

# one-hot encoding of categorical variables
app_train = pd.get_dummies(train_raw)
app_test = pd.get_dummies(test_raw)

train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)


app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))



poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')
poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

#Imputing missing value
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.fit_transform(poly_features_test)

#Polynomial transformation of degree 5

from sklearn.preprocessing import PolynomialFeatures

poly_transformer = PolynomialFeatures(degree = 5)


# Train polynomial features

poly_transformer.fit(poly_features)

# Transform the feature
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)

print('Poly train shape : ', poly_features.shape)
print('Poly test shape : ', poly_features_test.shape)


# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))
# Merge polynomial features into training dataframe
drop_columns = list(set(list(app_train.columns)).intersection(list(poly_features.columns)))
poly_features.drop(drop_columns, inplace=True, axis=1)

poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']

app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe

drop_columns_test = list(set(list(app_test.columns)).intersection(list(poly_features_test.columns)))
poly_features_test.drop(drop_columns_test, inplace=True, axis=1)

poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']

app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)



app_train_domain = app_train_poly.copy()
app_test_domain = app_test_poly.copy()

app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']



numeric_summary_table_train = pd.DataFrame(app_train_domain[list(app_train_domain.select_dtypes(['int64','float64']).columns)].describe().transpose())
numeric_summary_table_train['colnames'] = app_train_domain.select_dtypes(['int64','float64']).columns



numeric_summary_table_train_maxgt1 = numeric_summary_table_train[numeric_summary_table_train['max'] > 1]
std_columns_list = list(numeric_summary_table_train_maxgt1['colnames'])
columns_not_considered = ['SK_ID_CURR','CNT_CHILDREN']
std_columns_list = list(set(std_columns_list).difference(set(columns_not_considered)))


train_mod = app_train_domain.copy()
test_mod = app_test_domain.copy()
print (train_mod.shape)
print (test_mod.shape)


train_labels = train_mod['TARGET']
# Feature names
train_mod = train_mod.drop('TARGET',axis=1)
features = list(train_mod.columns)


from sklearn.preprocessing import MinMaxScaler, Imputer

# Median imputation of missing values
imputer = Imputer(strategy = 'median')


# Fit on the training data
imputer.fit(train_mod)

# Transform both training and testing data
train_mod = imputer.transform(train_mod)
test_mod = imputer.transform(test_mod)


# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))


# Repeat with the scaler
scaler.fit(train_mod)
train_mod = scaler.transform(train_mod)
test_mod = scaler.transform(test_mod)



print('Final train shape: ',train_mod.shape)
print('Final test shape: ',test_mod.shape)

3 columns were label encoded.
('Training Features shape: ', (3075, 216))
('Testing Features shape: ', (487, 215))
The non-anomalies default on 9.00% of loans
The anomalies default on 5.97% of loans
There are 553 anomalous days of employment
There are 97 anomalies in the test data out of 487 entries
('Poly train shape : ', (3075, 126))
('Poly test shape : ', (487, 126))
('Training data with polynomial features shape: ', (3075, 339))
('Testing data with polynomial features shape:  ', (487, 338))
(3075, 343)
(487, 342)
('Final train shape: ', (3075, 342))
('Final test shape: ', (487, 342))


In [14]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')

print ('Model run start')

voting_clf.fit(train_mod, train_labels)

print ('Model run complete')

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(train_mod, train_labels)
    train_pred = clf.predict(train_mod)
    print(clf.__class__.__name__, accuracy_score(train_labels,train_pred))

print ('Prediction dataset run Start')

test_pred = pd.DataFrame(voting_clf.predict(test_mod))
test_pred.to_csv('test_pred.csv')

print ('Final test prediction csv generated')

Model run start
Model run complete
('LogisticRegression', 0.9177235772357724)
('RandomForestClassifier', 0.9834146341463414)
('SVC', 0.9154471544715447)
('VotingClassifier', 0.92130081300813)
Prediction dataset run Start
Final test prediction csv generated


/Users/reetkanjilal/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/reetkanjilal/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [19]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [20]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_mod, train_labels)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=2000, min_samples_split=5, max_features=sqrt, max_depth=10 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=30, total=  31.6s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=2000, min_samples_split=5, max_features=sqrt, max_depth=10 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=30, total=  32.2s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=2000, min_samples_split=5, max_features=sqrt, max_depth=10 


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 13.4min


[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=90, total=  18.0s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=2000, min_samples_split=10, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=90, total=  17.3s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=2000, min_samples_split=10, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=2000, min_samples_split=10, max_features=sqrt, max_depth=10, total=  30.8s
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=1600, min_samples_split=5, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=2000, min_samples_split=10, max_features=sqrt, max_depth=10, total=  29.5s
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=1600, min_samples_split=5, max_features=sqrt, max_depth=10 
[CV]  bootstrap=Fa

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 64.4min


[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=10, max_features=auto, max_depth=50, total= 2.7min
[CV] bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=10, max_features=auto, max_depth=50 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=10, max_features=auto, max_depth=50, total= 3.1min
[CV] bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=5, max_features=auto, max_depth=30 
[CV]  bootstrap=False, min_samples_leaf=2, n_estimators=2000, min_samples_split=10, max_features=auto, max_depth=50, total=12.2min
[CV] bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=5, max_features=auto, max_depth=30 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=10, max_features=auto, max_depth=50, total=10.7min
[CV] bootstrap=False, min_samples_leaf=4, n_estimators=1000, min_samples_split=5, max_features=auto, max_depth=30 
[CV]  bootstrap

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 136.2min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [21]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 30,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 600}

In [ ]:
train

In [26]:
best_random = rf_random.best_estimator_
predictions = best_random.predict(train_mod)

predictions

In [28]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, train_mod, train_labels)

random_accuracy

Model Performance
Average Error: 0.0904 degrees.
Accuracy = -inf%.


-inf

In [29]:
le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels_class = le.transform(train_labels)